In [66]:
import geopandas as gpd

import numpy as np
import pandas as pd
from pandas import IndexSlice as idx
import tensorflow as tf
import sys
import os
import glob

from functools import partial



code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/perturbations/'
sys.path.append(code_dir)
code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/diff_bpr'
sys.path.append(code_dir)
from top_k import top_k_idx
#from make_datasets import make_data
from bpr_model import PerturbedBPRLinearModel, PerturbedBPRMLPModel
from make_datasets import make_data


code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/'
sys.path.append(code_dir)
from zinf_gp.metrics import normcdf, fixed_top_X



from perturbations import perturbed
from bpr import bpr_variable_k_no_ties



In [5]:
data_path='/cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/clean_quarter_tract/'

In [20]:
epochs = 5000
seed = 360
time_window = 5*4
timesteps_per_year = 4
first_train_eval_year = 2013
last_train_eval_year = 2017
batch_dim_size = (last_train_eval_year - first_train_eval_year + 1)*timesteps_per_year
validation_year = 2018
first_test_year = 2019
last_test_year = 2020

tf.random.set_seed(seed)


timestep_col = 'timestep'
geography_col = 'geoid'
outcome_col = 'deaths'

x_idx_cols = [geography_col, 'lat', 'lon', timestep_col,
              'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
              'svi_pctile', 'year',
              'neighbor_t', 'deaths']
y_idx_cols = [geography_col, timestep_col, outcome_col]
#features_only = ['lat', 'lon', timestep_col,
#                 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
#                 'svi_pctile',
#                 'neighbor_t', 'deaths']
features_only = ['deaths']

#data_gdf = gpd.read_file(data_path)

multiindexed_gdf = data_gdf.set_index(['geoid', 'timestep'])
multiindexed_gdf['timestep'] = multiindexed_gdf.index.get_level_values('timestep')
num_geoids = len(data_gdf['geoid'].unique())

train_shape = (num_geoids, time_window, len(features_only))

train_x_BSF_flat, train_y_BS = make_data(multiindexed_gdf, first_train_eval_year, last_train_eval_year,
                                                  time_window, features_only, train_shape, pred_lag=4)

valid_x_BSF_flat, valid_y_BS = make_data(multiindexed_gdf, validation_year, validation_year,
                                         time_window, features_only, train_shape, pred_lag=4)

test_x_BSF_flat, test_y_BS = make_data(multiindexed_gdf, first_test_year, last_test_year,
                                       time_window, features_only, train_shape, pred_lag=4)

norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(train_x_BSF_flat)
train_x_BSF_flat = norm_layer(train_x_BSF_flat)
valid_x_BSF_flat = norm_layer(valid_x_BSF_flat)
test_x_BSF_flat = norm_layer(test_x_BSF_flat)

top_100_idx_func = partial(top_k_idx, k=100)

In [21]:
perturbation_samples = 100
noise=0.2
learning_rate = 0.005


In [22]:
train_x_BSF_flat.shape

TensorShape([20, 1620, 24])

In [67]:
perturbed_top_100 = perturbed(top_100_idx_func,
                              num_samples=perturbation_samples,
                              sigma=noise,
                              noise='normal',
                              batched=True)

model = PerturbedBPRMLPModel(perturbed_top_k_func=perturbed_top_100,
                                hidden_sizes=[50, 10])

In [70]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Compile the model
def weird_loss(a, b):
    return -a / b

model.compile(optimizer=optimizer, loss=weird_loss)


In [71]:

model.fit(train_x_BSF_flat, train_y_BS, epochs=epochs, batch_size=20,
          validation_data=(valid_x_BSF_flat, valid_y_BS))

Epoch 1/5000
1/1 [==============================] - 1s 1s/step - loss: -0.1734 - val_loss: -0.1780
Epoch 2/5000
1/1 [==============================] - 0s 479ms/step - loss: -0.1682 - val_loss: -0.1742
Epoch 3/5000
1/1 [==============================] - 0s 479ms/step - loss: -0.1631 - val_loss: -0.1806
Epoch 4/5000
1/1 [==============================] - 0s 479ms/step - loss: -0.1674 - val_loss: -0.1782
Epoch 5/5000
1/1 [==============================] - 0s 480ms/step - loss: -0.1716 - val_loss: -0.1797
Epoch 6/5000


KeyboardInterrupt: 

In [19]:
first_validation_year

2019

In [26]:
from zinf_gp.metrics import normcdf, fixed_top_X

In [28]:
valid_y_BS[0]

<tf.Tensor: shape=(1620,), dtype=float32, numpy=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)>

In [34]:
fixed_top_X(np.mean(test_y_BS[4:],axis=0), test_y_BS,X=100)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'sort_values'

In [35]:
y_2019 = test_y_yr[test_y_yr[timestep_col]==19].groupby('geoid').sum()['deaths']
last_year = train_y[train_y[timestep_col] ==18].set_index('geoid')['deaths']
fixed_top_X(y_2019,last_year,100)[-1]

NameError: name 'test_y_yr' is not defined

In [61]:
bprs=[]
avg_2018 = multiindexed_gdf[(multiindexed_gdf['year']<=2018)&(multiindexed_gdf['year']>2013)].groupby(level='geoid')['deaths'].mean()
for quarter in [76,77,78,79]:
    this_2019 = multiindexed_gdf.loc[idx[:,quarter],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']
    bprs.append(fixed_top_X(this_2019,avg_2018,X=100)[-1])
np.mean(bprs)

0.39649931705176433

In [62]:
bprs=[]
avg_2018 = multiindexed_gdf[(multiindexed_gdf['year']<=2019)&(multiindexed_gdf['year']>2014)].groupby(level='geoid')['deaths'].mean()
for quarter in [80,81,82,83]:
    this_2019 = multiindexed_gdf.loc[idx[:,quarter],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']
    bprs.append(fixed_top_X(this_2019,avg_2018,X=100)[-1])
np.mean(bprs)

0.3802946487173787

In [64]:
bprs=[]
avg_2018 = multiindexed_gdf[(multiindexed_gdf['year']<=2020)&(multiindexed_gdf['year']>2015)].groupby(level='geoid')['deaths'].mean()
for quarter in [84,85,86,87]:
    this_2019 = multiindexed_gdf.loc[idx[:,quarter],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']
    bprs.append(fixed_top_X(this_2019,avg_2018,X=100)[-1])
np.mean(bprs)

0.38553360851793567

In [72]:
model.trainable_variables

[<tf.Variable 'perturbed_bprmlp_model/dense/kernel:0' shape=(24, 50) dtype=float32, numpy=
 array([[ 0.08130685, -0.26384673,  0.27236152, ...,  0.07897349,
         -0.2555355 ,  0.01948926],
        [ 0.09878462,  0.07647446, -0.24215457, ..., -0.22882546,
         -0.06209113,  0.05292066],
        [-0.21356046, -0.22734566,  0.17919381, ...,  0.17991634,
          0.16830131,  0.01721929],
        ...,
        [ 0.28220114,  0.01558735, -0.01301214, ..., -0.1351605 ,
         -0.19741191, -0.06966883],
        [-0.07399305,  0.00919886,  0.19608139, ...,  0.2191585 ,
          0.090666  , -0.17638008],
        [ 0.16001832,  0.07291204, -0.06433683, ..., -0.18710478,
         -0.04728039, -0.04247441]], dtype=float32)>,
 <tf.Variable 'perturbed_bprmlp_model/dense/bias:0' shape=(50,) dtype=float32, numpy=
 array([ 0.02550787, -0.00516292, -0.02024401, -0.02460235, -0.02564245,
         0.00287915, -0.00751161,  0.02415368, -0.00963079, -0.0253121 ,
         0.02115076,  0.00024868, 

In [74]:
model(train_x_BSF_flat)

<tf.Tensor: shape=(20, 1620), dtype=float32, numpy=
array([[-0.54194003, -0.49596536, -0.56974906, ..., -0.54194003,
        -0.54194003, -0.54194003],
       [-0.2910129 , -0.04409663, -0.30630913, ..., -0.2910129 ,
        -0.2910129 , -0.2910129 ],
       [-0.18490309, -0.7687056 , -0.31874678, ..., -0.18490309,
        -0.18490309, -0.18490309],
       ...,
       [-0.2005644 , -0.2005644 ,  0.02551813, ..., -0.02764977,
        -0.37251088, -0.2910129 ],
       [-0.5175134 , -0.5175134 , -0.02910141, ...,  0.2511227 ,
        -0.25267765, -0.18490309],
       [-0.08318724, -0.08318724,  0.21105245, ...,  0.19254676,
        -0.11209539, -0.10751391]], dtype=float32)>